In [25]:
import json

from prettytable import PrettyTable
from stringMatcher import levenshtein_ratio_and_distance
from time import time
table=PrettyTable(field_names=['date_first_available','mrp','image_url','product_url'])



# Load The database into the memory...


In [26]:
data = json.load(open("dataFinal_GIS.json", "rb"))

# Load the Global Secondary index in the memory...

In [27]:
GIS = json.load(open("kajal.json", "rb"))

# Write a function to search the product details by Pid

In [28]:
def getClosestMatch(queryString):
    # Initialize the time counter...
    tic                  = time()
    
    # Find closest match of the queryString from the Global Secondary Index...
    matchRatios          = [levenshtein_ratio_and_distance(queryString, KEY) for KEY, _ in GIS.items()]
    bestMatchRatio       = max(matchRatios)

    # If the best match ratio is less than 50% then we will assume that the records doesn't exist...
    if (bestMatchRatio < 0.5):
        return {
            "response code" : 404,
            "search time"   : f"{time()-tic} seconds",
            "message"       : "error",
            "response body" : "no match found"
        }
    
    # Otherwise...
    bestMatchRatio_index = matchRatios.index(bestMatchRatio)

    # Find the Pid of the movie which is the best match to the queryString...
    bestMatch_Pid = list(GIS)[bestMatchRatio_index]
    bestMatch_product_url = GIS[bestMatch_Pid]

    # Now we have Pid which is the primary key in our main data-store...
    # Retrieve the details of the product from the Pid
    dhamakaInfo            = data[str(bestMatch_product_url)]
    #try:
    #    dhamakaInfo=data[str(bestMatch_product_url)]
    #except KeyError:
    #    print("Not Found")
    
    # Mark the search completion time...
    toc                  = time()

    # Add Pid to the response...
    dhamakaInfo["Pid"] = bestMatch_Pid
    # Construct the response Model...
    responseModel = {
        "response code" : 200,
        "search time"   : f"{toc-tic} seconds",
        "message"       : "success",
        "response body" : dhamakaInfo
    }
    
    return responseModel

# Write a function to make a request and visualize the data returned by the search function..

In [29]:
def request(product=""):
    # product name should always be in a string...
    product = str(product)
    # Check if the Pid  is valid...
    if(product.replace(" ", "").strip() == ""):
        return f"Invalid Pid Name"
    
    # Make a request to fetch the product info...
    response = getClosestMatch(queryString = product)

    # Check the response code...
    if response["response code"] == 404:
        return f"ERROR: No records found"
    
   
    data_to_print = f"\
    date_first_available          : {response['response body']['date_first_available']}\n\
    Genre          : {response['response body']['genre']}\n\
    User Ratings   : ---------------------------\n\
    "
    print(data_to_print)
    print(table)

# Finally Make a request...

In [30]:
request(product="B085GL266P")

KeyError: "{'brandname': 'Bhagirath fab', 'datefirst': '2020-03-01T00:00:00', 'imageurl': 'https://images-na.ssl-images-amazon.com/images/I/61XinzsWunL._SL1100_.jpg', 'productdicrip': 'IND mask covers the user’s nose and mouth and provides a physical barrier to fluids and particulate materials. The surgical masks referenced in this guidance document include masks that are labeled as a surgical  laser  isolation  dental or medical procedure masks with or without a face shield.', 'productname': 'BHAGIRATH Standard comfort Disposable Pollution/Surgical Elastic Mask Surgical Face Mask with Earloop Great for Air Pollution Virus. 1', 'producturl': 'https://www.amazon.in/BHAGIRATH-Standard-Disposable-Pollution-Surgical/dp/B085GL266P/', 'rimi': {'discountprice': 0.99, 'mrp': 16999, 'saleprice': 199, 'sellername': 'Bhagirath fab'}}"

# the keyerror contains all info about the product user searched